In [1]:
import pandas as pd 
import numpy as np
import requests 
import constants
from pathlib import Path
import matplotlib.pyplot as plt

In /home/katie/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/katie/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/katie/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
data_path = constants.DENVER_POLICE_PATH
df = pd.read_csv(str(data_path))

df.head()

,INCIDENT_NUMBER,INCIDENT_DATE,INCIDENT_TIME,DAY_OF_WEEK,INITIAL_CONTACT,CONTACT_BASIS,STREET_ADDRESS,CITY_STATE,LAST_FIRST_NAME,ROLE,...,GENDER,AGE,RACE,ETHNICITY,ARMED_WITH,DISCHARGED_FIREARM,CASUALTY,DECISION_LETTER_URL,X_COORDINATE,Y_COORDINATE
0,20165000685.0,2016-02-22 00:00:00,1330.0,Monday,Citizen-Initiated,Burglary - In Progess,3500 blk N Lowell Blvd,"Denver, CO","Burke, Kevin",Officer,...,M,41.0,W,NH,Firearm,Yes,Not Injured,https://www.denverda.org/wp-content/uploads/de...,3130847.0,1704269.0
1,2016568888.0,2016-09-05 00:00:00,1207.0,Monday,Officer-Initiated,Warrant,300 blk N. Osage St.,"Denver, CO","Robledo, John",Officer,...,M,39.0,W,H,Firearm,Yes,Not Injured,None available,3139108.0,1688147.0
2,2019125830.0,2019-02-25 00:00:00,211.0,Monday,Citizen-Initiated,Disturbance,1200 N Galapago St,"Denver, CO","Litton, David",Subject,...,M,40.0,W,NH,Firearm,No,Deceased,https://www.denverda.org/wp-content/uploads/20...,3141672.0,1693222.0
3,201516702.0,2015-01-09 00:00:00,1940.0,Friday,Officer-Initiated,Vehicle Stop,5081 N Crown Blvd,"Denver, CO","Kindell, Sharod",Subject,...,M,24.0,B,NH,Motor Vehicle,NaN,Injured,http://www.denverda.org/News_Release/Decision_...,3188620.0,1712709.0
4,201960655.0,2019-01-27 00:00:00,1141.0,Sunday,Citizen-Initiated,Shots Fired,622 N Inca St,"Denver, CO","Lee, Eric",Officer,...,M,41.0,W,NH,Firearm,Yes,Not Injured,Investigation Pending,3141423.0,1689770.0


In [3]:
df.columns


Index(['INCIDENT_NUMBER', 'INCIDENT_DATE', 'INCIDENT_TIME', 'DAY_OF_WEEK',
       'INITIAL_CONTACT', 'CONTACT_BASIS', 'STREET_ADDRESS', 'CITY_STATE',
       'LAST_FIRST_NAME', 'ROLE', 'OFFICER_RANK', 'BADGE_NUMBER',
       'DUTY_STATUS', 'UNIFORM_PLAINCLOTHES', 'GENDER', 'AGE', 'RACE',
       'ETHNICITY', 'ARMED_WITH', 'DISCHARGED_FIREARM', 'CASUALTY',
       'DECISION_LETTER_URL', 'X_COORDINATE', 'Y_COORDINATE'],
      dtype='object')

In [4]:
# from 2015 to 2020
df.INCIDENT_DATE.max(), df.INCIDENT_DATE.min()

('2020-01-01 00:00:00', '2015-01-09 00:00:00')

In [5]:
# only 45 officer involved shootings 
df.INCIDENT_NUMBER.nunique()

45

In [6]:
# 4 officer involved shootings w/o a decision letter
num_none_available = df.INCIDENT_NUMBER.loc[df.DECISION_LETTER_URL=='None available'].nunique()

# 6 officier involved shootings where investigation is still pending
num_investigation_pending = df.INCIDENT_NUMBER.loc[df.DECISION_LETTER_URL=='Investigation Pending'].nunique()

# that leaves 35 incidents

# drop them from the following dataset
df_subset = df.loc[(df.DECISION_LETTER_URL!='None available') & (df.DECISION_LETTER_URL!='Investigation Pending')]

# make a df for status code of decision letters
df_status_code = df_subset[['INCIDENT_NUMBER', 'DECISION_LETTER_URL']]
df_status_code.drop_duplicates(inplace=True)

df_status_code['STATUS_CODE'] = df_status_code.DECISION_LETTER_URL.apply(lambda url: requests.get(url).status_code)

# drop 3 rows where the page cannot be found '404'
df_status_code = df_status_code.loc[df_status_code.STATUS_CODE != 404]

# down to 34 incidents w decision letters 
num_incident = df_status_code.INCIDENT_NUMBER.nunique()


/home/katie/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/katie/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [7]:
# # download the available decision letters
# decision_letters_path = constants.DECISION_LETTERS_PATH

# for index, row in df_status_code.iterrows():
#     url = row.DECISION_LETTER_URL
#     incident_num = row.INCIDENT_NUMBER
#     r = requests.get(url)
#     pdf_path = Path.joinpath(decision_letters_path, str(incident_num)+".pdf")

#     with open(pdf_path,'wb') as f: 
#         f.write(r.content) 

In [8]:
grouped = df.CONTACT_BASIS.groupby(df.INCIDENT_NUMBER)
grouped.unique()

INCIDENT_NUMBER
2015153192.0                                             [Warrant]
201516702.0                                         [Vehicle Stop]
2015288243.0                           [Weapon / Concealed Weapon]
2015482538.0                                        [Vehicle Stop]
201549266.0                                   [Suspicious Vehicle]
20155002522.0                          [Weapon / Concealed Weapon]
20155004253.0                          [Weapon / Concealed Weapon]
2015701574.0                                             [Warrant]
2015713662.0                                        [Vehicle Stop]
2015726322.0                              [Burglary - In Progress]
201621883.0                                        [Vehicle Check]
2016226732.0                                             [Warrant]
2016398914.0                     [Domestic Violence - In Progress]
2016435311.0                                            [Shooting]
2016484812.0                                  